# End Station Finder

## The challenge:
My friend Lou has a project. They want to visit all the end stations on the Berlin S-Bahn and U-Bahn system. However, to count as a truly satisfying end station, the station must not only be at the end of its own line, but must also lack an interchange to another line. Lou wants only the end-iest of end stations.

I want to find the end stations and present them to Lou in a format they can use for their project.


## The data:

Berlin provides public transport data in the now-standard [GTFS](https://gtfs.org/) format, which stands for General (formerly Google) Transit Feed Specification. It is an [open standard](https://beyondtransparency.org/chapters/part-2/pioneering-open-data-standards-the-gtfs-story/) used by transit agencies to present their data in a uniform und useable way. It encompasses two types of data: realtime and static.
- The realtime data is what drives (pun intended) our on-the-go transit apps and notifies us of delays and changes.
- The static data (also known as *schedule data*) tells us and our apps about how all the points on the network are linked together. This is the GTFS data that will allow us identify the end stations.

### Static GTFS data format
GTFS is a data format in its own right, meant for "[GTFS-consuming applications](https://www.transitwiki.org/TransitWiki/index.php/Category:GTFS-consuming_applications)". However, conveniently, it is also easily readable in other software environements because it is merely a ZIP comprised of CSV (as TXT) files. The exact tables provided vary by agency. The following are the minimum required [table files for static GTFS](https://gtfs.org/schedule/reference/):
-  `stops.txt` showing stops' name and location
-  `stop_times.txt` showing every stop on every individual trip
-  `trips.txt` showing trips' route, direction, destination* (headsign), and more
-  `routes.txt` showing routes' transportation type, number, line colour, and more
-  `agency.txt`

My [GTFS download for Berlin](https://daten.berlin.de/datensaetze/vbb-fahrplandaten-gtfs) also contains these conditionally required and optional tables:
-  `frequencies.txt` (note: empty), `calendar.txt` and `calendar_dates.txt` showing service frequency, day and date availability
-  `pathways.txt` and `levels.txt` showing platform positions inside stations
-  `shapes.txt` showing a trip's path, expressed as a sequence of co-ordinates 
-  `transfers.txt` showing the connections between stations and details about these transfers

GTFS does not contain a simple list of `stops` on `route`. Instead, `stop_ID`s appear as sequences of `stop_times` in individual `trips`, which in turn belong to various `routes`. We need to join the IDs of stops, trips and routes to the (giant) `stop_times.txt`. 

I would therefore need to combine the following tables and fields to find the end stations if I were to make a database:

`stop_times.txt` for the sequence of stops
- "trip_id"         (primary key 1) (foreign key of trips.trip_id)
- "stop_sequence"   (primary key 2)
- "arrival_time"
- "departure_time"
- "stop_id" (foreign key of stops.stop_id)
- "pickup_type"
- "drop_off_type"

`transfers.txt` for the transfers between different routes (or in our case, the lack thereof)

- "from_stop_id"    (primary key 1) (foreign key of stops.stop_id)
- "to_stop_id"      (primary key 2) (foreign key 2 of stops.stop_id)
- "from_route_id"   (primary key 3) - might not be needed
- "to_route_id"     (primary key 4) - might not be needed
- "transfer_type"`  see [docs](https://gtfs.org/schedule/reference/#transferstxt)
- "min_transfer_time" 

`trips.txt` for matching the trip IDs to routes
- "trip_id"         (primary key)
- "route_id"        (foreign key of routes.route_id)
- "trip_headsign"   *
- "trip_short_name" (can be route number but appears not to be used for train numbers here)
- "direction_id"

`stops.txt` for matching the stop IDs to stops
- "stop_id"         (primary key)
- "stop_name"       
- "stop_lon"
- "stop_lat"

\* the "headsign" destination is not to be confused with the end of a line; some trips do not travel the whole route. In our data this is a text field rather than an ID.

## Tools for GTFS

There are many [packages designed to handle GTFS data](https://gtfs.org/resources/gtfs) in Python, created by various developers. These are mainly used for geographical and frequency data, but I found three that I may be able to use to find the end stations in **Python**, more easily than by manually creating a database:

|    | Package | Data Structure | Potential Functions |
| ---| ------- | -------------- | ------------------- |
| 1.    | [`gtfs_kit`](https://mrcagney.github.io/gtfs_kit_docs/)   | Pandas  | `gtfs_kit.stops.get_stops` and `gtfs_kit.validators.check_transfers`|
| 2. | [`transit_service_analyst`](https://github.com/psrc/transit_service_analyst/wiki/transit_service_analyst-documentation) | Pandas | `get_line_stops_gdf()` |
| 3. | [`gtfs_utils`](https://open-bus-gtfs-utils.readthedocs.io/en/latest/route_stats.html)    | Pandas |  `route_stats` and `trip_stats`| 

I also noted down a few tools for **SQL**, in case the above don't work for me: 
1. [`node-gtfs`](https://github.com/BlinkTagInc/node-gtfs) SQLite
2. [`gtfs-via-postgres`](https://github.com/public-transport/gtfs-via-postgres) PostgreSQL
3. [`gtfs-schema`](https://github.com/tyleragreen/gtfs-schema) Schema builder
4. [`gtfsdb`](https://github.com/OpenTransitTools/gtfsdb) Works with various SQL


## Load data: attempt with `transit_service_analyst`
### Step 1: Load packages, extensions and data

In [45]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import plotly.express as px
import shapely.geometry
import numpy as np

I decided to install and try the `transit_service_analyst` package first. It also requires `geopandas` and `pandera` to be installed. Now I can import the module:

In [2]:
import transit_service_analyst as tsa

Then I used the command `tsa.load_gtfs(folder, service_date)` to try to load the data into a Pandas dataframe. The required arguments are:
- the location of the unzipped GTFS folder
- a sample date from the dataset in YYYYMMDD form. 

A date is needed because `tsa` constructs the routes based on the stops served in all scheduled trips on the sample day. A look in `calendar.txt` shows that my data runs from 20230615 to 20231209, so I picked today, 20230705, a Wednesday.

**Note on date and route sampling issues:**
Choosing only a single date to build the routes is a potential issue because there is the possibility that it might not sample the whole length of routes. For example, in the case of construction work that prevents trains reaching their normal end station, the scheduled trip data might show an point earlier than the end of the physical route. This can be partially controlled for using information already contained in Berlin's GTFS data. Indeed, today, like most or maybe all days, is marked as an exception in `calendar_dates.txt`. Later I will check the affected `service_id`s to find out if they are relevant. However, this would not account for any shortened routes that span enough time to be counted as the normal schedule rather than the exception, and I will attempt to overcome this later, probably by sampling various dates from the set.

In [ ]:
transit_analyst = tsa.load_gtfs('Berlin GTFS data', '20230705')

In [18]:
#i got an error: `Cannot set a DataFrame with multiple columns to the single column start_time_secs` [and other _time_secs columns]
#attempting to debug and reload. 
#Autoreload lets me use the changes instantly without a kernel reload.
%load_ext autoreload
%autoreload 2

In [4]:
transit_analyst = tsa.load_gtfs('Berlin GTFS data', '20230705')
#routes_df failed Pandera schema validation and yielded another error so I added lazy=True to all dfs' validation in the source 

/home/sian/anaconda3/envs/stats_env/lib/python3.9/site-packages/transit_service_analyst/gtfs_service.py:95: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  trips_df = pd.read_csv(os.path.join(self.gtfs_dir, "trips.txt"))
/home/sian/anaconda3/envs/stats_env/lib/python3.9/site-packages/transit_service_analyst/gtfs_service.py:116: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times_df = pd.read_csv(os.path.join(self.gtfs_dir, "stop_times.txt"))


SchemaErrors: Schema Routes: A total of 1 schema errors were found.

Error Counts
------------
- SchemaErrorReason.SCHEMA_COMPONENT_CHECK: 1

Schema Error Summary
--------------------
                                                                                    failure_cases  n_failure_cases
schema_context column     check                                                                                   
Column         route_type isin([0, 1, 2, 3, 4, 5, 6, 7, 11, 12])  [700, 100, 900, 1000, 109, 400]                6

Usage Tip
---------

Directly inspect all errors by catching the exception:

```
try:
    schema.validate(dataframe, lazy=True)
except SchemaErrors as err:
    err.failure_cases  # dataframe of schema errors
    err.data  # invalid dataframe
```


### Failures due to data compatibility issues

#### Empty csv
Unfortunately it seems like this package requires a populated `frequencies.txt` to gather the `trip_id`s. This csv is however empty in Berlin's static data. The code in `gtfs_service.py` was set up to deal with the file existing or not existing, but not being empty:

```python
if os.path.exists(os.path.join(self.gtfs_dir, "frequencies.txt")):
            self.trips, self.stop_times = self.frequencies_to_trips()
```
Solved for now by renaming the file, but it would be good to suggest a change on Github later.

#### Inconsistent `route_type`s
Berlin's data uses [Google Transit extended route types](https://developers.google.com/transit/gtfs/reference/extended-route-types) rather than the classic [GTFS route types](https://gtfs.org/schedule/reference/#routestxt) so Pandera's schema validation fails:

```console
    Error Counts
    ------------
    - SchemaErrorReason.SCHEMA_COMPONENT_CHECK: 1
    
    Schema Error Summary
    --------------------
                                                                                        failure_cases  n_failure_cases
    schema_context column     check                                                                                   
    Column         route_type isin([0, 1, 2, 3, 4, 5, 6, 7, 11, 12])  [700, 100, 900, 1000, 109, 400]                6
```

Can I easily update the schema?

## Load data: attempt with [`gtfs_kit`](https://mrcagney.github.io/gtfs_kit_docs/)
### Step 1: Load packages, extensions and data

I installed `gtfs_kit` in my conda envoronment and imported it. This module uses Pandas and Shapely to manipulate the GTFS data.

In [8]:
import gtfs_kit as gk

In [9]:
gk.list_feed('Berlin GTFS data')

,file_name,file_size
0,pathways.txt,8946320
1,trips.txt,16658440
2,transfers.txt,3321256
3,agency.txt,3255
4,stops.txt,7201534
5,routes.txt,48522
6,shapes.txt,163661343
7,levels.txt,6098
8,0frequencies.txt,64
9,calendar_dates.txt,874177


This loads our GTFS data into an instance of the module's `Feed` class, ignoring non-GTFS files and stripping whitespaces from column names as it does so.

In [10]:
#ref: https://github.com/mrcagney/gtfs_kit/blob/master/notebooks/examples.ipynb
feed = gk.read_feed('Berlin GTFS data', dist_units='km')

The `Feed` class has a specialised `describe()` method which shows us information about the component `DataFrame`s of our instance:

In [11]:
feed.describe()

,indicator,value
0,agencies,"[S-Bahn Berlin GmbH, Oberhavel Verkehrsgesells..."
1,timezone,Europe/Berlin
2,start_date,20230615
3,end_date,20231209
4,num_routes,1257
5,num_trips,238544
6,num_stops,41151
7,num_shapes,14515
8,sample_date,20230622
9,num_routes_active_on_sample_date,1190


### Step 2: List stops

In [20]:
#I'm not sure which modules I need to import as well here...
import shapely as shp
import pyproj
import geopandas

In [83]:
berlin_routes = feed.get_routes()

Knowing that the Berlin data is using Extended GTFS journey types, I will extract the routes directly by number:
- **109**: Suburban Railway.	Examples:	S-Bahn (DE)
- **400**: Urban Railway Service   (note: not 402: Underground Service, despite U-Bahn being an example given)

In [51]:
berlin_routes[(berlin_routes.route_type == 109) | (berlin_routes.route_type == 400)]

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color,route_desc
151,21649_109,1,S25,NaN,109,NaN,NaN,NaN
152,18950_109,1,S3,NaN,109,NaN,NaN,NaN
153,18949_109,1,S26,NaN,109,NaN,NaN,NaN
154,16814_109,1,S7,NaN,109,NaN,NaN,NaN
156,12003_109,1,S85,NaN,109,NaN,NaN,NaN
157,11561_109,1,S3,NaN,109,NaN,NaN,NaN
158,11537_109,1,S46,NaN,109,NaN,NaN,NaN
159,11536_109,1,S46,NaN,109,NaN,NaN,NaN
165,10281_109,1,S8,NaN,109,NaN,NaN,NaN
166,10277_109,1,S1,NaN,109,NaN,NaN,NaN


Here we can see that the type is also appended in the `route_id`.

However, it's not immedately obvious how to link the routes and the individual stops:

In [65]:
feed.get_stops().head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,platform_code,zone_id,level_id
0,de:11000:900100007::3,NaN,S Oranienburger Str. (Berlin),Ersatzhalt Tucholskystraße vor Oranienburger S...,52.524724,13.392833,0,NaN,0,NaN,5555 S Oranienburger Str. (Berlin),4.0
1,de:12070:900215110:1:50,NaN,"Bad Wilsnack, Bahnhof",Bahnsteig Gleis 2,52.960114,11.949402,0,de:12070:900215110,1,2,"4533 Bad Wilsnack, Bahnhof",50.0
2,de:12070:900215110:2:51,NaN,"Bad Wilsnack, Bahnhof",Bahnsteig Gleis 3,52.960219,11.949528,0,de:12070:900215110,1,3,"4533 Bad Wilsnack, Bahnhof",50.0
3,de:12062:900415465:1:50,NaN,"Prösen, Bahnhof",Bahnsteig Gleis 1,51.434919,13.488216,0,de:12062:900415465,1,1,"7958 Prösen, Bahnhof",4.0
4,de:12062:900415465:2:51,NaN,"Prösen, Bahnhof",Bahnsteig Gleis 2,51.434886,13.488277,0,de:12062:900415465,1,2,"7958 Prösen, Bahnhof",4.0


In [80]:
#does not work..
# tt = feed.build_route_timetable(route_id='17512', dates='20230707')
# tt

""


`stop_times` has a trip_id, stop_id and stop_sequence, but we would still need to join the routes and the number of connections. at this point there seems to be no advantage over importing it myself.

In [86]:
feed.stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_headsign
0,200164528,5:33:00,5:33:00,de:12068:900205112::1,0,0,0,NaN
1,200164528,5:34:00,5:34:00,de:12068:900205155::1,1,0,0,NaN
2,200164528,5:35:00,5:35:00,de:12068:900205110::1,2,0,0,NaN
3,200164528,5:37:00,5:37:00,de:12068:900205576::1,3,0,0,NaN
4,200164528,5:38:00,5:38:00,de:12068:900205577::1,4,0,0,NaN
...,...,...,...,...,...,...,...,...
5476106,209074811,7:12:00,7:12:00,de:12064:900320850::1,6,0,0,NaN
5476107,209074811,7:13:00,7:13:00,de:12064:900320843::2,7,0,0,NaN
5476108,209074811,7:15:00,7:15:00,de:12064:900320832::4,8,0,0,NaN
5476109,209074811,7:34:00,7:34:00,de:12064:900320553::1,9,0,0,NaN


## Load data: attempt with SQL or Pandas?